# Semantic Segmentation with PyTorch

Mount google drive to colab.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import neccessary libraties and set parameters.

In [2]:
import os
import time
import json

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models

In [3]:
project_name = "HW3"
project_path = "/content/drive/My Drive/" + project_name
train_dataset_path = project_path + "/SimulationDataset/train"
test_dataset_path = project_path + "/SimulationDataset/test"

model_type = "unet" # encdec / fcn / unet / pspnet

# Create folder to store training results.
if model_type == "encdec":
    results_path = project_path + "/results_encdec"
elif model_type == "fcn":
    results_path = project_path + "/results_fcn"
elif model_type == "unet":
    results_path = project_path + "/results_unet"
elif model_type == "pspnet":
    results_path = project_path + "/results_pspnet"

if os.path.isdir(results_path) == False:
   os.mkdir(results_path)

# Parameters
num_class = 3 
input_h, input_w = 256, 256
batch_size = 16
epochs = 10
lr = 1e-4
use_gpu = torch.cuda.is_available()

## Simulation Dataset

In [4]:
class SimDataset(Dataset):
    def __init__(self, path, n_class=num_class, flip_rate=0.5, train=True):
        self.img_folder_path = path + "/img"
        self.label_folder_path = path + "/label"
        self.file_list = os.listdir(self.img_folder_path)
        self.n_class = n_class
        self.flip_rate = flip_rate
        self.train = train

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.img_folder_path + "/" + self.file_list[idx]
        label_path = self.label_folder_path + "/" + "jetbot_" + self.file_list[idx].split("_")[1] + "_layer.png"

        # open image data
        img = np.asarray(Image.open(img_path).resize((256, 256), Image.NEAREST))
        img = img.astype(float)/255.0
        label_img = np.asarray(Image.open(label_path).resize((256, 256), Image.NEAREST))
        label = np.zeros((img.shape[0], img.shape[1], self.n_class), dtype=float)
        label[label_img[:,:,0]==178,0] = 1
        label[label_img[:,:,0]==255,1] = 1
        label[label_img[:,:,0]==0,2] = 1

        if np.random.sample() < self.flip_rate:
            img = np.fliplr(img)
            label = np.fliplr(label)

        img = torch.from_numpy(img.copy()).float()
        img = img.permute(2,0,1)
        label = torch.from_numpy(label.copy()).float()
        label = label.permute(2,0,1)
        sample = {"X": img, "Y": label}
        return sample

# Load dataset
train_data = SimDataset(path=train_dataset_path, flip_rate=0.5)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
test_data = SimDataset(path=test_dataset_path, flip_rate=0.0)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

## Network Model
### VGG16 Feature Extractor (pretrained)

In [5]:
class Vgg16(nn.Module):
    def __init__(self, pretrained = True):
        super(Vgg16, self).__init__()
        self.vggnet = models.vgg16(pretrained)
        del(self.vggnet.classifier) # Remove fully connected layer to save memory.
        features = list(self.vggnet.features)
        self.layers = nn.ModuleList(features).eval() 
        
    def forward(self, x):
        results = []
        for ii,model in enumerate(self.layers):
            x = model(x)
            if ii in [3,8,15,22,29]:
                results.append(x) #(64,256,256),(128,128,128),(256,64,64),(512,32,32),(512,16,16)
        return results

vgg_model = Vgg16()
vgg_model = vgg_model.cuda()
print(vgg_model.layers)

ModuleList(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

### Encoder-Decoder

In [6]:
class DeConv2d(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size, stride, padding, dilation):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode="nearest")
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation)
    
    def forward(self, x):
        output = self.up(x)
        output = self.conv(output)
        return output

class EncoderDecoder(nn.Module):
    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        self.relu = nn.ReLU(inplace=True)

        self.deconv1 = DeConv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn1 = nn.BatchNorm2d(512)
        
        self.deconv2 = DeConv2d(512, 256, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn2 = nn.BatchNorm2d(256)
        
        self.deconv3 = DeConv2d(256, 128, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.deconv4 = DeConv2d(128, 64, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        self.classifier = nn.Conv2d(64, n_class, kernel_size=1)

    def forward(self, x):
        pre_output = self.pretrained_net(x)
        output = self.bn1(self.relu(self.deconv1(pre_output[4]))) #(512,32,32)
        output = self.bn2(self.relu(self.deconv2(output))) #(256,64,64)
        output = self.bn3(self.relu(self.deconv3(output))) #(128,128,128)
        output = self.bn4(self.relu(self.deconv4(output))) #(64,256,256)
        output = self.classifier(output)
        return output

### Fully Convolution Network (FCN)


In [7]:
class FCN(nn.Module):
    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        self.relu = nn.ReLU(inplace=True)
        #####################################
        #TODO
        #####################################
        self.deconv1 = DeConv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn1 = nn.BatchNorm2d(512)
        
        self.deconv2 = DeConv2d(512, 256, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn2 = nn.BatchNorm2d(256)
        
        self.deconv3 = DeConv2d(256, 128, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.deconv4 = DeConv2d(128, 64, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        self.classifier = nn.Conv2d(64, n_class, kernel_size=1)

    def forward(self, x):
        #####################################
        #TODO
        #####################################
        pre_output = self.pretrained_net(x)
        output = self.bn1(self.relu(self.deconv1(pre_output[4])))   # (512, 32, 32)
        output = torch.add(output, pre_output[3])
        output = self.bn2(self.relu(self.deconv2(output)))          # (256, 64, 64)
        output = torch.add(output, pre_output[2])
        output = self.bn3(self.relu(self.deconv3(output)))          # (128, 128, 128)
        output = torch.add(output, pre_output[1])
        output = self.bn4(self.relu(self.deconv4(output)))          # (64, 256, 256)
        output = torch.add(output, pre_output[0])
        output = self.classifier(output)
        return output

### U-Net

In [8]:
class UNet(nn.Module):
    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        self.relu = nn.ReLU(inplace=True)
        #####################################
        #TODO
        #####################################
        self.deconv1 = DeConv2d(512, 512, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn1 = nn.BatchNorm2d(512)
        
        self.deconv2 = DeConv2d(512 * 2, 256, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn2 = nn.BatchNorm2d(256)
        
        self.deconv3 = DeConv2d(256 * 2, 128, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.deconv4 = DeConv2d(128 * 2, 64, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        self.classifier = nn.Conv2d(64 * 2, n_class, kernel_size=1)
    
    def forward(self, x):
        #####################################
        #TODO
        #####################################
        pre_output = self.pretrained_net(x)
        output = self.bn1(self.relu(self.deconv1(pre_output[4])))   # (512, 32, 32)
        # pre_output_3 = self.bn1(self.relu(self.deconv1(pre_output[3])))
        output = torch.cat([output, pre_output[3]], dim=1)
        output = self.bn2(self.relu(self.deconv2(output)))          # (256, 64, 64)
        # pre_output_2 = self.bn1(self.relu(self.deconv1(pre_output[2])))
        output = torch.cat([output, pre_output[2]], dim=1)
        output = self.bn3(self.relu(self.deconv3(output)))          # (128, 128, 128)
        # pre_output_1 = self.bn1(self.relu(self.deconv1(pre_output[1])))
        output = torch.cat([output, pre_output[1]], dim=1)
        output = self.bn4(self.relu(self.deconv4(output)))          # (64, 256, 256)
        # pre_output_0 = self.bn1(self.relu(self.deconv1(pre_output[0])))
        output = torch.cat([output, pre_output[0]], dim=1)
        output = self.classifier(output)
        return output

### PSPNet

In [9]:
class PSPNet(nn.Module):
    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        #####################################
        #TODO
        #####################################
        self.relu = nn.ReLU(inplace=True)

        self.ppm_size = (16, 16)
        self.ppm_channel = 512
        self.ppm_psize = [1, 2, 3, 6]

        self.ppm_pool, self.ppm_conv, self.ppm_up = [], [], []
        for psize in self.ppm_psize:
            self.ppm_pool.append(nn.AdaptiveAvgPool2d((psize, psize)))
            self.ppm_conv.append(nn.Conv2d(int(self.ppm_channel), int(self.ppm_channel/len(self.ppm_psize)), kernel_size=1))
            self.ppm_up.append(nn.Upsample(size=self.ppm_size, mode='bilinear', align_corners=True))
        
        self.ppm_pool = nn.ModuleList(self.ppm_pool)
        self.ppm_conv = nn.ModuleList(self.ppm_conv)
        self.ppm_up = nn.ModuleList(self.ppm_up)

        self.deconv1 = DeConv2d(1024, 512, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn1 = nn.BatchNorm2d(512)
        
        self.deconv2 = DeConv2d(512, 256, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn2 = nn.BatchNorm2d(256)
        
        self.deconv3 = DeConv2d(256, 128, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.deconv4 = DeConv2d(128, 64, kernel_size=3, stride=1, padding=1, dilation=1)
        self.bn4 = nn.BatchNorm2d(64)
        
        self.classifier = nn.Conv2d(64, n_class, kernel_size=1)

    def forward(self, x):
        #####################################
        #TODO
        #####################################
        pre_output = self.pretrained_net(x)
        ppm_list = [pre_output[4]]
        for i in range(len(self.ppm_psize)):
            output = self.ppm_pool[i](pre_output[4])
            output = self.ppm_conv[i](output)
            output = self.ppm_up[i](self.relu(output))
            ppm_list.append(output)
        output = torch.cat(ppm_list, 1)
        output = self.bn1(self.relu(self.deconv1(output)))          # (512, 32, 32)
        output = torch.add(output, pre_output[3])
        output = self.bn2(self.relu(self.deconv2(output)))          # (256, 64, 64)
        output = torch.add(output, pre_output[2])
        output = self.bn3(self.relu(self.deconv3(output)))          # (128, 128, 128)
        output = torch.add(output, pre_output[1])
        output = self.bn4(self.relu(self.deconv4(output)))          # (64, 256, 256)
        output = torch.add(output, pre_output[0])
        output = self.classifier(output)

        return output

Construct models.

In [10]:
if model_type == "encdec":
    seg_model = EncoderDecoder(pretrained_net=vgg_model, n_class=num_class)
elif model_type == "fcn":
    seg_model = FCN(pretrained_net=vgg_model, n_class=num_class)
elif model_type == "unet":
    seg_model = UNet(pretrained_net=vgg_model, n_class=num_class)
elif model_type == "pspnet":
    seg_model = PSPNet(pretrained_net=vgg_model, n_class=num_class)

seg_model = seg_model.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(seg_model.parameters(), lr=lr)

## Training and Validation

In [11]:
def train(seg_model, train_loader, test_loader):
    # pixel accuracy and mIOU list 
    pixel_acc_list = []
    mIOU_list = []
    for epoch in range(1, epochs+1):
        ts = time.time()
        for iter, batch in enumerate(train_loader):
            optimizer.zero_grad()
            inputs, labels = batch["X"], batch["Y"]
            if use_gpu:
              inputs = inputs.cuda()
              labels = labels.cuda()

            outputs = seg_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch:{:2}, iter:{:2}, loss: {:.4f}".format(epoch, iter, loss.data.item()))
        
        print("Finish epoch:{:2}, time elapsed: {:.4f}".format(epoch, time.time() - ts))
        
        print("Start evaluation ...")
        acc, iou = eval(seg_model, test_loader)
        pixel_acc_list.append(acc)
        mIOU_list.append(iou)

        print("Output test results ...")
        file_name = results_path + "/" + str(epoch).zfill(3) + ".jpg"
        for iter, batch in enumerate(test_loader):
          inputs, labels = batch["X"], batch["Y"]
          if use_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()
          outputs = seg_model(inputs)
          save_result(file_name, inputs, labels, outputs)
          break
        
        print("Save model ...")
        model_path = results_path + "/" + "segnet.pt"
        torch.save(seg_model.state_dict(), model_path)
        print("========================================")
        
    highest_pixel_acc = max(pixel_acc_list)
    highest_mIOU = max(mIOU_list)
    
    highest_pixel_acc_epoch = pixel_acc_list.index(highest_pixel_acc)
    highest_mIOU_epoch = mIOU_list.index(highest_mIOU)
    
    # Extract evaluation record
    record_path = results_path + "/record.json"
    ret = json.dumps({"acc":pixel_acc_list, "iou":mIOU_list})
    with open(record_path, 'w') as fp:
        fp.write(ret)
    
    print("The highest mIOU is {} and is achieved at epoch-{}".format(highest_mIOU, highest_mIOU_epoch+1))
    print("The highest pixel accuracy  is {} and is achieved at epoch-{}".format(highest_pixel_acc, highest_pixel_acc_epoch+1))

In [12]:
def eval(seg_model, test_loader):
    seg_model.eval()
    total_ious = []
    pixel_accs = []

    for iter, batch in enumerate(test_loader): ## batch is 1 in this case
        inputs = torch.FloatTensor(batch["X"])
        if use_gpu:
          inputs = inputs.cuda()

        output = seg_model(inputs)
        
        # only save the 1st image for comparison
        if iter == 0:
            # generate images
            input_np = batch["X"][0].data.cpu().numpy().transpose(1,2,0)
            output_np = output[0].data.cpu().numpy().transpose(1,2,0)
            gt_np = batch["Y"][0].data.cpu().numpy().transpose(1,2,0)
        output = output.data.cpu().numpy()

        N, _, h, w = output.shape
        pred = output.transpose(0, 2, 3, 1).reshape(-1, num_class).argmax(axis=1).reshape(N, h, w)
        target = batch['Y'].data.cpu().numpy().transpose(0, 2, 3, 1).reshape(-1, num_class).argmax(axis=1).reshape(N, h, w)

        for p, t in zip(pred, target):
            total_ious.append(iou(p, t))
            pixel_accs.append(pixel_acc(p, t))

    # Calculate average IoU
    total_ious = np.array(total_ious).T  # n_class * val_len
    ious = np.nanmean(total_ious, axis=1)
    pixel_accs = np.array(pixel_accs).mean()
    print("pix_acc: {:.4f}, meanIoU: {:.4f}".format(pixel_accs, np.nanmean(ious)))
    return pixel_accs, np.nanmean(ious)

# Calculates class intersections over unions
def iou(pred, target):
    ious = []
    for cls in range(num_class):
        pred_inds = pred == cls
        target_inds = target == cls
        intersection = pred_inds[target_inds].sum()
        union = pred_inds.sum() + target_inds.sum() - intersection
        if union == 0:
            ious.append(float("nan")) # if there is no ground truth, do not include in evaluation
        else:
            ious.append(float(intersection) / max(union, 1))
    return ious

def pixel_acc(pred, target):
    correct = (pred == target).sum()
    total = (target == target).sum()
    return correct / total

def save_result(file_name, input, label, output, n_samples=3):
    input_np = input[:n_samples].data.cpu().numpy().transpose(0,2,3,1)
    label_np = label[:n_samples].data.cpu().numpy().transpose(0,2,3,1)
    output_np = output[:n_samples].data.cpu().numpy().transpose(0,2,3,1)
    
    result_list = []
    for k in range(n_samples):
        tmp = np.zeros([256,256,3], dtype=np.float32)
        for i in range(256):
            for j in range(256):
                tmp[i,j,output_np[k][i,j].argmax()] = 1
        result = np.hstack((input_np[k], label_np[k], tmp))
        result_list.append(result)

    # horizontally stack original image and its corresponding segmentation results
    vstack_image = np.vstack(result_list)
    new_im = Image.fromarray(np.uint8(vstack_image*255))
    new_im.save(file_name)

In [13]:
# perform training 
train(seg_model, train_loader, test_loader)

epoch: 1, iter: 0, loss: 0.7302
epoch: 1, iter:10, loss: 0.3157
epoch: 1, iter:20, loss: 0.2540
epoch: 1, iter:30, loss: 0.2194
epoch: 1, iter:40, loss: 0.1918
epoch: 1, iter:50, loss: 0.1677
epoch: 1, iter:60, loss: 0.1480
epoch: 1, iter:70, loss: 0.1326
epoch: 1, iter:80, loss: 0.1188
epoch: 1, iter:90, loss: 0.1178
epoch: 1, iter:100, loss: 0.0977
epoch: 1, iter:110, loss: 0.0911
epoch: 1, iter:120, loss: 0.0833
Finish epoch: 1, time elapsed: 306.8669
Start evaluation ...
pix_acc: 0.9913, meanIoU: 0.9841
Output test results ...
Save model ...
epoch: 2, iter: 0, loss: 0.0806
epoch: 2, iter:10, loss: 0.0388
epoch: 2, iter:20, loss: 0.0159
epoch: 2, iter:30, loss: 0.0158
epoch: 2, iter:40, loss: 0.0113
epoch: 2, iter:50, loss: 0.0086
epoch: 2, iter:60, loss: 0.0068
epoch: 2, iter:70, loss: 0.0087
epoch: 2, iter:80, loss: 0.0083
epoch: 2, iter:90, loss: 0.0048
epoch: 2, iter:100, loss: 0.0051
epoch: 2, iter:110, loss: 0.0053
epoch: 2, iter:120, loss: 0.0049
Finish epoch: 2, time elapsed

In [14]:
# Used for evaluation
#load_path = results_path + "/" + "segnet.pt"
#seg_model.load_state_dict(torch.load(load_path))
#eval(seg_model, test_loader)